In [ ]:
"""
This code was taken from:

Danzi, M.C., Dohrn, M.F., Fazal, S. et al. 
Deep structured learning for variant prioritization in Mendelian diseases. 
Nat Commun 14, 4167 (2023). https://doi.org/10.1038/s41467-023-39306-7
"""

In [1]:
resources = '../../resources/'

In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO

## Inheritance map

In [3]:
genemap=pd.read_csv(resources + 'genemap2_20240702.txt',sep='\t',low_memory=False,comment='#')
genemap = genemap.rename(columns={'Approved Gene Symbol': 'Approved Symbol'})
genemap=genemap.loc[:,['MIM Number','Approved Symbol','Ensembl Gene ID','Phenotypes']]
genemap=genemap.loc[~genemap['Approved Symbol'].isna(),:]
genemap=genemap.loc[~genemap['Phenotypes'].isna(),:]
genemap

,MIM Number,Approved Symbol,Ensembl Gene ID,Phenotypes
24,147571,ISG15,ENSG00000187608,"Immunodeficiency 38, 616126 (3), Autosomal rec..."
25,103320,AGRN,ENSG00000188157,"Myasthenic syndrome, congenital, 8, with pre- ..."
30,600315,TNFRSF4,ENSG00000186827,"?Immunodeficiency 16, 615593 (3), Autosomal re..."
32,615291,B3GALT6,ENSG00000176022,"Ehlers-Danlos syndrome, spondylodysplastic typ..."
36,611354,INTS11,ENSG00000127054,Neurodevelopmental disorder with motor and lan...
...,...,...,...,...
18427,300777,TMLHE,ENSG00000185973,"{Autism, susceptibility to, X-linked 6}, 30087..."
18441,400020,SHOX,ENSG00000185960,"Short stature, idiopathic familial, 300582 (3)..."
18450,480000,SRY,ENSG00000184895,"46XY sex reversal 1, 400044 (3), Y-linked; 46X..."
18456,400033,TBL1Y,ENSG00000092377,"?Deafness, Y-linked 2, 400047 (3), Y-linked"


In [4]:
mimToGene=genemap.loc[:,['MIM Number','Approved Symbol']]
phenotypes=pd.DataFrame(genemap['Phenotypes'].str.split(';').tolist(), index=genemap['MIM Number'].values).stack().str.strip()
phenotypes

147571  0    Immunodeficiency 38, 616126 (3), Autosomal rec...
103320  0    Myasthenic syndrome, congenital, 8, with pre- ...
600315  0    ?Immunodeficiency 16, 615593 (3), Autosomal re...
615291  0    Ehlers-Danlos syndrome, spondylodysplastic typ...
        1    Spondyloepimetaphyseal dysplasia with joint la...
                                   ...                        
400020  2    Leri-Weill dyschondrosteosis, 127300 (3), Pseu...
480000  0            46XY sex reversal 1, 400044 (3), Y-linked
        1    46XX sex reversal 1, 400045 (4), X-linked domi...
400033  0          ?Deafness, Y-linked 2, 400047 (3), Y-linked
400005  0    Spermatogenic failure, Y-linked, 2, 415000 (3)...
Length: 7568, dtype: object

In [5]:
filter=((phenotypes.str[0]=="?") | (phenotypes.str[0]=="[") | (phenotypes.str[0]=="{"))
phenotypes2=phenotypes.loc[~filter,:]
phenotypes2
filter2=phenotypes2.str.contains(r" [0-9]{6} \(")
phenotypes2=phenotypes2.loc[filter2,:]
phenotypes2

147571  0    Immunodeficiency 38, 616126 (3), Autosomal rec...
103320  0    Myasthenic syndrome, congenital, 8, with pre- ...
615291  0    Ehlers-Danlos syndrome, spondylodysplastic typ...
        1    Spondyloepimetaphyseal dysplasia with joint la...
        2    Al-Gazali syndrome, 609465 (3), Autosomal rece...
                                   ...                        
400020  1    Langer mesomelic dysplasia, 249700 (3), Pseudo...
        2    Leri-Weill dyschondrosteosis, 127300 (3), Pseu...
480000  0            46XY sex reversal 1, 400044 (3), Y-linked
        1    46XX sex reversal 1, 400045 (4), X-linked domi...
400005  0    Spermatogenic failure, Y-linked, 2, 415000 (3)...
Length: 6046, dtype: object

In [6]:
locusMIMs=phenotypes2.index.get_level_values(0)
phenotypeMIMs=phenotypes2.str.split(r"\([0-9]\)").str[0].str.split(',').str[-1].str.strip()
phenotypeInheritances=phenotypes2.str.split(r"\([0-9]\), ").str[1].str.strip()
mappingMethod=phenotypes2.str.split(r" [0-9]{6} \(").str[1].str.split(')').str[0]

In [7]:
newMap=pd.DataFrame({'geneMIM':locusMIMs,'phenoMIM':phenotypeMIMs.values,'inheritance':phenotypeInheritances.values,'mappingMethod':mappingMethod.values})
newMap

,geneMIM,phenoMIM,inheritance,mappingMethod
0,147571,616126,Autosomal recessive,3
1,103320,615120,Autosomal recessive,3
2,615291,615349,Autosomal recessive,3
3,615291,271640,Autosomal recessive,3
4,615291,609465,Autosomal recessive,3
...,...,...,...,...
6041,400020,249700,Pseudoautosomal recessive,3
6042,400020,127300,Pseudoautosomal dominant,3
6043,480000,400044,Y-linked,3
6044,480000,400045,X-linked dominant,4


In [8]:
newMap.to_csv('./geneToPhenoToInheritanceMap.txt',sep='\t',index=False)

In [9]:
newMap2=newMap.loc[((newMap['inheritance']=="Autosomal dominant") | (newMap['inheritance']=="Autosomal recessive")),:]
newMap3=newMap2.loc[newMap2['mappingMethod']=='3',:]
# add back in the gene names
mimToGene=mimToGene.rename(columns={'MIM Number':'geneMIM','Approved Symbol':'GeneSymbol'})
newMap4=newMap3.merge(mimToGene,how='inner',on='geneMIM')
newMap4.to_csv('./geneToPhenoToInheritanceMap_filtered.txt',sep='\t',index=False)

## ClinVar variants

In [8]:
inheritanceMap=pd.read_csv('./geneToPhenoToInheritanceMap_filtered.txt',sep='\t',low_memory=False)
inheritanceMap=inheritanceMap.drop_duplicates(subset=['geneMIM','phenoMIM','inheritance'],keep='first')
inheritanceMap

,geneMIM,phenoMIM,inheritance,mappingMethod,GeneSymbol
0,147571,616126,Autosomal recessive,3,ISG15
1,103320,615120,Autosomal recessive,3,AGRN
2,615291,615349,Autosomal recessive,3,B3GALT6
3,615291,271640,Autosomal recessive,3,B3GALT6
4,615291,609465,Autosomal recessive,3,B3GALT6
...,...,...,...,...,...
5074,604272,604377,Autosomal recessive,3,SCO2
5075,131222,603041,Autosomal recessive,3,TYMP
5076,612395,602541,Autosomal recessive,3,CHKB
5077,607574,250100,Autosomal recessive,3,ARSA


In [9]:
clinvar=pd.read_csv(resources + 'variant_summary_2025-01.txt',sep='\t',low_memory=False)
clinvar.loc[:,'Assembly'].value_counts()

Assembly
GRCh37    3110359
GRCh38    3059494
na           6931
NCBI36       4771
Name: count, dtype: int64

In [10]:
clinvar = clinvar.loc[clinvar['Assembly'] == 'GRCh38',:]

In [11]:
len(clinvar.loc[clinvar['PositionVCF'] == -1])

19277

In [15]:
clinvar = clinvar.loc[clinvar['PositionVCF'] != -1]

In [12]:
clinvar.head()

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),...,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF,SomaticClinicalImpact,SomaticClinicalImpactLastEvaluated,ReviewStatusClinicalImpact,Oncogenicity,OncogenicityLastEvaluated,ReviewStatusOncogenicity
1,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,-,397704705,...,2,4781213,GGAT,TGCTGTAAACTGTAACTGTAAA,-,-,-,-,-,-
3,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,3,4787729,GCTGCTGGACCTGCC,G,-,-,-,-,-,-
5,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,...,4,84799209,G,A,-,-,-,-,-,-
7,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,1,"Dec 01, 2023",267606829,...,5,126275389,C,T,-,-,-,-,-,-
9,15045,single nucleotide variant,NM_017547.4(FOXRED1):c.1289A>G (p.Asn430Ser),55572,FOXRED1,HGNC:26927,Pathogenic,1,"Oct 01, 2010",267606830,...,6,126277517,A,G,-,-,-,-,-,-


In [17]:
clinvar = clinvar.rename(columns={'#AlleleID': 'AlleleID'})
clinvar.loc[:,'Type'].value_counts()

Type
single nucleotide variant    2793322
Deletion                      127178
Duplication                    57487
Microsatellite                 33695
Indel                          14927
Insertion                      11948
Inversion                       1288
Variation                        372
Name: count, dtype: int64

In [16]:
clinvar=clinvar.loc[((clinvar['Type']=="single nucleotide variant") | (clinvar['Type']=="Deletion") | (clinvar['Type']=="Duplication") | (clinvar['Type']=="Microsatellite") | (clinvar['Type']=="Indel") | (clinvar['Type']=="Insertion")),:]
benign=clinvar.loc[((clinvar['ClinicalSignificance']=="Benign") | (clinvar['ClinicalSignificance']=="Likely benign") | (clinvar['ClinicalSignificance']=="Benign/Likely benign")),:]
benign=benign.loc[~(benign['OriginSimple']=="somatic"),:]
pathogenic=clinvar.loc[((clinvar['ClinicalSignificance']=="Pathogenic") | (clinvar['ClinicalSignificance']=="Likely pathogenic") | (clinvar['ClinicalSignificance']=="Pathogenic/Likely pathogenic")),:]
pathogenic=pathogenic.loc[~(clinvar['OriginSimple']=="somatic"),:]
pathogenic=pathogenic.loc[pathogenic['PhenotypeIDS'].str.contains("OMIM:"),:]

In [17]:
# get the relationships between clinvar allele IDs and omim phenotype ids 
pathogenicOmimPheno=pd.DataFrame(pathogenic['PhenotypeIDS'].str.split(';').tolist(), index=pathogenic['AlleleID'].values).stack()
pathogenicOmimPheno=pathogenicOmimPheno[pathogenicOmimPheno.str.contains("OMIM:")]
pathogenicAlleleIDs=pathogenicOmimPheno.index.get_level_values(0)
pathogenicAlleleOmimIDs=pathogenicOmimPheno.str.split("OMIM:").str[1].str.split(",").str[0].str.strip()
pathogenicAllelesToOmimIDs=pd.DataFrame({'clinvarAlleleID':pathogenicAlleleIDs,'phenoMIM':pathogenicAlleleOmimIDs.values})

In [18]:
# select down to just the allele IDs for which we have at least one inheritance pattern in omim
pathogenicAllelesWithInheritance=pathogenicAllelesToOmimIDs.merge(inheritanceMap,how='inner',on='phenoMIM')
# select down to just the variants without conflicts in the inheritance
pathogenicAllelesWithInheritance2=pathogenicAllelesWithInheritance.drop_duplicates(subset=['clinvarAlleleID','inheritance'],keep='first')
pathogenicAllelesWithInheritance3=pathogenicAllelesWithInheritance2.drop_duplicates(subset='clinvarAlleleID',keep=False)
# get the original data on the alleles with omim inheritance
pathogenic2=pathogenic.merge(pathogenicAllelesWithInheritance3,how='inner',left_on=['AlleleID'],right_on=['clinvarAlleleID'])
pathogenic2.loc[:,'ReviewStatus'].value_counts()

ReviewStatus
criteria provided, single submitter                     71577
criteria provided, multiple submitters, no conflicts    36377
no assertion criteria provided                          17339
reviewed by expert panel                                 2662
practice guideline                                          3
Name: count, dtype: int64

In [19]:
pathogenic3=pathogenic2.loc[~(pathogenic2['ReviewStatus']=="no assertion criteria provided"),:]
len(pathogenic3.loc[pathogenic3['inheritance']=='Autosomal recessive',:]) # this is the number of recessive variants we have now (but this includes coding and noncoding)

66869

In [20]:
len(pathogenic3.loc[pathogenic3['inheritance']=='Autosomal dominant',:]) # this is the number of dominant variants we have now (but this includes coding and noncoding)

43750

In [21]:
pathogenic3.columns

Index(['AlleleID', 'Type', 'Name', 'GeneID', 'GeneSymbol_x', 'HGNC_ID',
       'ClinicalSignificance', 'ClinSigSimple', 'LastEvaluated', 'RS# (dbSNP)',
       'nsv/esv (dbVar)', 'RCVaccession', 'PhenotypeIDS', 'PhenotypeList',
       'Origin', 'OriginSimple', 'Assembly', 'ChromosomeAccession',
       'Chromosome', 'Start', 'Stop', 'ReferenceAllele', 'AlternateAllele',
       'Cytogenetic', 'ReviewStatus', 'NumberSubmitters', 'Guidelines',
       'TestedInGTR', 'OtherIDs', 'SubmitterCategories', 'VariationID',
       'PositionVCF', 'ReferenceAlleleVCF', 'AlternateAlleleVCF',
       'SomaticClinicalImpact', 'SomaticClinicalImpactLastEvaluated',
       'ReviewStatusClinicalImpact', 'Oncogenicity',
       'OncogenicityLastEvaluated', 'ReviewStatusOncogenicity',
       'clinvarAlleleID', 'phenoMIM', 'geneMIM', 'inheritance',
       'mappingMethod', 'GeneSymbol_y'],
      dtype='object')

In [22]:
pathogenic3AR=pathogenic3.loc[pathogenic3['inheritance']=='Autosomal recessive',['Chromosome','PositionVCF','ReferenceAlleleVCF','AlternateAlleleVCF']]
pathogenic3AD=pathogenic3.loc[pathogenic3['inheritance']=='Autosomal dominant',['Chromosome','PositionVCF','ReferenceAlleleVCF','AlternateAlleleVCF']]
pathogenic3AR=pathogenic3AR.rename(columns={'PositionVCF':'Start','ReferenceAlleleVCF':'ReferenceAllele','AlternateAlleleVCF':'AlternateAllele'})
pathogenic3AD=pathogenic3AD.rename(columns={'PositionVCF':'Start','ReferenceAlleleVCF':'ReferenceAllele','AlternateAlleleVCF':'AlternateAllele'})
pathogenic3AR=pathogenic3AR.sort_values(by=['Chromosome','Start','ReferenceAllele','AlternateAllele'])
pathogenic3AD=pathogenic3AD.sort_values(by=['Chromosome','Start','ReferenceAllele','AlternateAllele'])
pathogenic3AR.to_csv('./clinvar_pathogenic_AR_locations.txt',sep='\t',index=False)
pathogenic3AD.to_csv('./clinvar_pathogenic_AD_locations.txt',sep='\t',index=False)

In [23]:
# deal with clinvar benigns
benign=benign.loc[~(benign['ReviewStatus']=="no assertion criteria provided"),:]
benign=benign.loc[:,['Chromosome','PositionVCF','ReferenceAlleleVCF','AlternateAlleleVCF']]
benign=benign.rename(columns={'PositionVCF':'Start','ReferenceAlleleVCF':'ReferenceAllele','AlternateAlleleVCF':'AlternateAllele'})
benign.to_csv('./clinvar_oneToFourStarBenign_locations.txt',sep='\t',index=False)

In [24]:
# Add gnomAD benigns 
gnomad=pd.read_csv(resources + 'gnomad.exomes.v4.1.sites.benign.txt',sep='\t',low_memory=False)
benign2=pd.concat([benign,gnomad],axis=0,ignore_index=True)
benign2=benign2.sort_values(by=['Chromosome','Start','ReferenceAllele','AlternateAllele'])
benign2=benign2.drop_duplicates(subset=['Chromosome','Start','ReferenceAllele','AlternateAllele'],keep='first').reset_index(drop=True)
benign2.to_csv('./clinvar_benign_locations.txt',sep='\t',index=False)
len(benign2) # this is the number of benign variants we have now

2614673

- `dos2unix`: converts the line breaks from the format `Carriage Return + Line Feed` into the format `Line Feed`  
- `tail -n +2`: reads the whole file except for the first line  
- `awk -F'\t' -v OFS='\t' '{print $1,$2,".",$3,$4,".","PASS",".","GT","0/1"}'`: reads a tab-separated, processes each line from the format `Chrom Start Ref Alt` to the format `Chrom Start . Ref Alt . PASS . GT 0/1`, and returns another tab-separated file
- `cat header.vcf`: adds VCF header (28 lines)
- `convert2annovar.pl` converts VCF to annovar input format
- `annotate_variation.pl` annotates the fule acording to the build and database (here, Gencode is used)
- `coding_change.pl` converts DNA sequence into protein

In [25]:
%%bash 
cd ../../resources/
path="../training2025/training_data2025_ann2025_hg38/"
dos2unix "$path"clinvar_oneToFourStar*
tail -n +2 "$path"clinvar_pathogenic_AR_locations.txt | awk -F'\t' -v OFS='\t' '{print $1,$2,".",$3,$4,".","PASS",".","GT","0/1"}' | cat header.vcf - > "$path"clinvar_pathogenic_AR.vcf
tail -n +2 "$path"clinvar_pathogenic_AD_locations.txt | awk -F'\t' -v OFS='\t' '{print $1,$2,".",$3,$4,".","PASS",".","GT","0/1"}' | cat header.vcf - > "$path"clinvar_pathogenic_AD.vcf
tail -n +2 "$path"clinvar_benign_locations.txt | awk -F'\t' -v OFS='\t' '{print $1,$2,".",$3,$4,".","PASS",".","GT","0/1"}' | cat header.vcf - > "$path"clinvar_benign.vcf
                                                           
annovar/convert2annovar.pl -format vcf4 "$path"clinvar_pathogenic_AR.vcf > "$path"clinvar_pathogenic_AR.avinput
annovar/annotate_variation.pl -dbtype wgEncodeGencodeBasicV47 -buildver hg38 --exonicsplicing "$path"clinvar_pathogenic_AR.avinput annovar/humandb/
annovar/convert2annovar.pl -format vcf4 "$path"clinvar_pathogenic_AD.vcf > "$path"clinvar_pathogenic_AD.avinput
annovar/annotate_variation.pl -dbtype wgEncodeGencodeBasicV47 -buildver hg38 --exonicsplicing "$path"clinvar_pathogenic_AD.avinput annovar/humandb/
annovar/convert2annovar.pl -format vcf4 "$path"clinvar_benign.vcf > "$path"clinvar_benign.avinput
annovar/annotate_variation.pl -dbtype wgEncodeGencodeBasicV47 -buildver hg38 --exonicsplicing "$path"clinvar_benign.avinput annovar/humandb/

annovar/coding_change.pl "$path"clinvar_pathogenic_AR.avinput.exonic_variant_function annovar/humandb/hg38_wgEncodeGencodeBasicV47.txt annovar/humandb/hg38_wgEncodeGencodeBasicV47Mrna.fa --includesnp --onlyAltering --alltranscript > "$path"clinvar_pathogenic_AR.coding_changes.txt
annovar/coding_change.pl "$path"clinvar_pathogenic_AD.avinput.exonic_variant_function annovar/humandb/hg38_wgEncodeGencodeBasicV47.txt annovar/humandb/hg38_wgEncodeGencodeBasicV47Mrna.fa --includesnp --onlyAltering --alltranscript > "$path"clinvar_pathogenic_AD.coding_changes.txt
annovar/coding_change.pl "$path"clinvar_benign.avinput.exonic_variant_function annovar/humandb/hg38_wgEncodeGencodeBasicV47.txt annovar/humandb/hg38_wgEncodeGencodeBasicV47Mrna.fa --includesnp --onlyAltering --alltranscript > "$path"clinvar_benign.coding_changes.txt

cd "$path"

dos2unix: converting file ../training2025/training_data2025_ann2025_hg38_test_known2020/clinvar_oneToFourStarBenign_locations.txt to Unix format...
NOTICE: Finished reading 66897 lines from VCF file
NOTICE: A total of 66869 locus in VCF file passed QC threshold, representing 38898 SNPs (22911 transitions and 15987 transversions) and 27886 indels/substitutions
NOTICE: Finished writing 38898 SNP genotypes (22911 transitions and 15987 transversions) and 27886 indels/substitutions for 1 sample
NOTICE: The --geneanno operation is set to ON by default
NOTICE: Output files are written to ../training2025/training_data2025_ann2025_hg38_test_known2020/clinvar_pathogenic_AR.avinput.variant_function, ../training2025/training_data2025_ann2025_hg38_test_known2020/clinvar_pathogenic_AR.avinput.exonic_variant_function
NOTICE: Reading gene annotation from annovar/humandb/hg38_wgEncodeGencodeBasicV47.txt ... Done with 154259 transcripts (including 80867 without coding sequence annotation) for 63750 uniq

In [3]:
approvedTranscripts=pd.read_csv(resources + 'gencodeBasicFullLengthTranscriptsConversionTable_v47.txt',sep='\t',low_memory=False)

canonical=pd.read_csv(resources + 'gnomad.v4.1.constraint_simple_canonical.tsv',sep='\t',low_memory=False)
# remove the gnomad canonical transcripts that are not approvedTranscripts
canonical=canonical.loc[canonical['transcript'].isin(approvedTranscripts['transcriptIDShort'].values),:].reset_index(drop=True)

GTEx=pd.read_csv(resources + 'GTEx.v10.medians.tsv',sep='\t',low_memory=False)
# remove the non-approvedTranscripts from the expression data
GTEx=GTEx.loc[GTEx['transcript_id'].isin(approvedTranscripts['transcriptIDShort'].values),:].reset_index(drop=True)
# add a overall expression column
GTEx['overallAvg']=GTEx.iloc[:,2:55].mean()

sequences={}
for record in SeqIO.parse(resources + "gencode.v47.pc_translations.fa","fasta"):
    transcriptID=record.id.split('|')[1][:15]
    if len(approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,:])>0:
        sequences[transcriptID]=record.seq

def groomAnnovarOutput(base,sequences=sequences,approvedTranscripts=approvedTranscripts,canonical=canonical,GTEx=GTEx):

	sample=pd.read_csv(base + ".avinput.exonic_variant_function",sep='\t',low_memory=False,header=None,
						names=['line','varType','location','hg38_chr','hg38_pos(1-based)','end','ref','alt','genotype','qual','depth'],
                       dtype={'line':str,'varType':str,'location':str,'hg38_chr':str,'hg38_pos(1-based)':np.int32,'end':np.int32,'ref':str,'alt':str,'genotype':str,'qual':str,'depth':str})
	# add new columns with placeholders to be filled in
	sample['WildtypeSeq']=""
	sample['AltSeq']=""
	sample['ChangePos']=-1
	sample['TranscriptID']=""
	sample['TranscriptIDShort']=""
	sample['geneName']=""
	sample['geneID']=""
	sample['geneIDShort']=""
	

	for i in range(len(sample)):
		if i % 1000 == 0:
			print(str(i) + ' rows completed')
		numTranscripts=len(sample.loc[i,'location'].split(','))
		numCanonical=0
		canonicals=[]
		transcripts=[]
		transcriptLengths=[]
		canonicalTranscript=""
		correctedGeneName=""
		for j in range(numTranscripts-1):
			if sample.loc[i,'location'].split(',')[j].split(':')[1][:15] in canonical['transcript'].values:
				numCanonical=numCanonical+1
				canonicals.append(sample.loc[i,'location'].split(',')[j].split(':')[1][:15])
			if sample.loc[i,'location'].split(',')[j].split(':')[1][:15] in approvedTranscripts['transcriptIDShort'].values:  
				transcripts.append(sample.loc[i,'location'].split(',')[j].split(':')[1][:15])
				transcriptLengths.append(len(sequences[sample.loc[i,'location'].split(',')[j].split(':')[1][:15]]))
				
		if len(transcripts)>0:
			if numCanonical==1:
				transcriptID=canonicals[0]
				sample.loc[i,'TranscriptIDShort']=transcriptID
				sample.loc[i,'TranscriptID']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'transcriptID'].values[0]
				sample.loc[i,'geneName']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneName'].values[0]
				sample.loc[i,'geneID']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneID'].values[0]
				sample.loc[i,'geneIDShort']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneIDShort'].values[0]
			elif numCanonical==0:
				if len(transcripts)==1:
					transcriptID=transcripts[0]
					sample.loc[i,'TranscriptIDShort']=transcriptID
					sample.loc[i,'TranscriptID']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'transcriptID'].values[0]
					sample.loc[i,'geneName']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneName'].values[0]
					sample.loc[i,'geneID']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneID'].values[0]
					sample.loc[i,'geneIDShort']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneIDShort'].values[0]
				else:
					if len(GTEx.loc[GTEx['transcript_id'].isin(transcripts),:])>0:
						# pick the transcript with the highest expression
						transcriptID=GTEx.loc[GTEx['transcript_id'].isin(transcripts),:].sort_values(by=['overallAvg'],ascending=False).reset_index(drop=True).iloc[0,0]
						sample.loc[i,'TranscriptIDShort']=transcriptID
						sample.loc[i,'TranscriptID']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'transcriptID'].values[0]
						sample.loc[i,'geneName']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneName'].values[0]
						sample.loc[i,'geneID']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneID'].values[0]
						sample.loc[i,'geneIDShort']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneIDShort'].values[0]
					else:
						# if none of the transcripts have measured expression and none of them are canonical, then pick the one with the longest amino acid sequence
						# if multiple tie for longest, this picks the one we saw first
						j=transcriptLengths.index(max(transcriptLengths))
						transcriptID=transcripts[j]
						sample.loc[i,'TranscriptIDShort']=transcriptID
						sample.loc[i,'TranscriptID']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'transcriptID'].values[0]
						sample.loc[i,'geneName']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneName'].values[0]
						sample.loc[i,'geneID']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneID'].values[0]
						sample.loc[i,'geneIDShort']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneIDShort'].values[0]
			elif numCanonical>1:
				if len(GTEx.loc[GTEx['transcript_id'].isin(canonicals),:])>0:
					# pick the canonical transcript with the highest expression
					transcriptID=GTEx.loc[GTEx['transcript_id'].isin(canonicals),:].sort_values(by=['overallAvg'],ascending=False).reset_index(drop=True).iloc[0,0]
					sample.loc[i,'TranscriptIDShort']=transcriptID
					sample.loc[i,'TranscriptID']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'transcriptID'].values[0]
					sample.loc[i,'geneName']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneName'].values[0]
					sample.loc[i,'geneID']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneID'].values[0]
					sample.loc[i,'geneIDShort']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneIDShort'].values[0]
				else:
					# if none of the canonical transcripts have measured expression, then pick the one with the longest amino acid sequence
					# if multiple tie for longest, this picks the one we saw first
					j=transcriptLengths.index(max(transcriptLengths))
					transcriptID=transcripts[j]
					sample.loc[i,'TranscriptIDShort']=transcriptID
					sample.loc[i,'TranscriptID']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'transcriptID'].values[0]
					sample.loc[i,'geneName']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneName'].values[0]
					sample.loc[i,'geneID']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneID'].values[0]
					sample.loc[i,'geneIDShort']=approvedTranscripts.loc[approvedTranscripts['transcriptIDShort']==transcriptID,'geneIDShort'].values[0]

	# drop the entries without transcript ID matches
	sample=sample.loc[(~(sample['TranscriptIDShort']=="")),:].reset_index(drop=True)

	for record in SeqIO.parse(base + ".coding_changes.txt", "fasta"):
		lineNum=record.id
		# only use the transcript that we selected above 
		if ((len(sample.loc[sample['line']==lineNum,:])>0) and (sample.loc[sample['line']==lineNum,'TranscriptIDShort'].values[0]==record.description.split(' ')[1][:15])):
			if 'WILDTYPE' in record.description:
				if record.seq.__str__()[:-1] == sequences[record.description.split(' ')[1][:15]]:
					sample.loc[sample['line']==lineNum,'WildtypeSeq']=record.seq.__str__()
			else:
				sample.loc[sample['line']==lineNum,'AltSeq']=record.seq.__str__()
				if 'startloss' in record.description:
					sample.loc[sample['line']==lineNum,'ChangePos']=1
				elif 'silent' in record.description:
					sample.loc[sample['line']==lineNum,'ChangePos']=-1
				else:
					sample.loc[sample['line']==lineNum,'ChangePos']=record.description.split(' ')[7].split('-')[0]
	sample2=sample.loc[~((sample['WildtypeSeq']=="") | (sample['AltSeq']=="") | (sample['ChangePos']==-1)),:]
	sample2.to_csv(base + '.groomed.txt',sep='\t',index=False)
	return


In [ ]:
groomAnnovarOutput('clinvar_pathogenic_AR')
groomAnnovarOutput('clinvar_pathogenic_AD')
groomAnnovarOutput('clinvar_benign')

0 rows completed
1000 rows completed
2000 rows completed


In [ ]:
constraint=pd.read_csv(resources + 'gnomad.v4.1.constraint_simple.tsv',sep='\t',low_memory=False)

gnomadAF=pd.read_csv(resources + 'gnomad410_GRCh38_exomes_AFs.txt',sep='\t',low_memory=False,dtype={'hg38_chr':str,'hg38_pos(1-based)':np.int32,'ref':str,'alt':str,'AF':np.float32,'nhomalt':np.int32,'controls_AF':np.float32,'controls_nhomalt':np.int32})
gnomadAF=gnomadAF.loc[(~(gnomadAF['hg38_chr'].str.contains('_'))),:].reset_index(drop=True)
gnomadAF.loc[gnomadAF['hg38_chr']=='X','hg38_chr']=23
gnomadAF.loc[gnomadAF['hg38_chr']=='Y','hg38_chr']=24
gnomadAF.loc[gnomadAF['hg38_chr']=='MT','hg38_chr']=25
gnomadAF['hg38_chr']=gnomadAF['hg38_chr'].astype(int)

CCR=pd.read_csv(resources + 'ccrs_GRCh38.enumerated.txt',sep='\t',low_memory=False,dtype={'chrom':str,'pos':np.int32,'ccr_pct':np.float32})
CCR=CCR.loc[(~(CCR['chrom'].str.contains('_'))),:]
CCR.loc[CCR['chrom']=='X','chrom']=23
CCR['chrom']=CCR.loc[:,'chrom'].astype(int)
CCR=CCR.sort_values(by=['chrom','pos','ccr_pct'],ascending=[True,True,False]).drop_duplicates(subset=['chrom','pos'],keep='first').reset_index(drop=True)

pext=pd.read_csv(resources + 'gnomAD_pext_values.gtex.v10.txt',sep='\t',low_memory=False,dtype={'chr':str,'pos':np.int32,'pext':np.float32})
pext=pext.loc[(~(pext['chr'].str.contains('_'))),:]
pext.loc[pext['chr']=='X','chr']=23
pext.loc[pext['chr']=='Y','chr']=24
pext.loc[pext['chr']=='M','chr']=25
pext['chr']=pext.loc[:,'chr'].astype(int)
pext=pext.sort_values(by=['chr','pos','pext'],ascending=[True,True,False]).drop_duplicates(subset=['chr','pos'],keep='first').reset_index(drop=True)

gerp=pd.read_csv(resources + 'gerpOnExons_GRCh38.txt',sep='\t',low_memory=False,dtype={'chr':str,'pos':np.int32,'gerp':np.float32})
gerp=gerp.loc[(~(gerp['chr'].str.contains('_'))),:]
gerp.loc[gerp['chr']=='X','chr']=23
gerp.loc[gerp['chr']=='Y','chr']=24
gerp.loc[gerp['chr']=='M','chr']=25
gerp['chr']=gerp['chr'].astype(int)
gerp=gerp.sort_values(by=['chr','pos','gerp'],ascending=[True,True,False]).drop_duplicates(subset=['chr','pos'],keep='first').reset_index(drop=True)

GDI=pd.read_csv(resources + 'GDI.groomed.txt',sep='\t',low_memory=False)
RVIS=pd.read_csv(resources + 'RVIS.groomed.txt',sep='\t',low_memory=False)

In [ ]:
def annotateVariantsAndFilter(base,constraint=constraint,gnomadAF=gnomadAF,CCR=CCR,pext=pext,gerp=gerp,GDI=GDI,RVIS=RVIS,variantType='normal'):
    sample=pd.read_csv(base + '.groomed.txt',sep='\t',low_memory=False)
    sample.loc[sample['hg38_chr']=='X','hg38_chr']=23
    sample.loc[sample['hg38_chr']=='Y','hg38_chr']=24
    sample.loc[sample['hg38_chr']=='MT','hg38_chr']=25
    sample['hg38_chr']=sample['hg38_chr'].astype(int)

    # merge on the allele frequency data
    sample=sample.merge(gnomadAF,how='left',on=['hg38_chr','hg38_pos(1-based)','ref','alt'])

    # merge on the constraint data (try transcript ID merge first)
    sampleTranscript=sample.merge(constraint,how='inner',left_on=['TranscriptIDShort'],right_on=['transcript'])
    notMatched=sample.loc[~(sample['TranscriptIDShort'].isin(sampleTranscript['TranscriptIDShort'])),:]
    sampleGeneID=notMatched.merge(constraint,how='inner',left_on=['geneIDShort'],right_on=['gene_id'])
    notMatched2=notMatched.loc[~(notMatched['geneIDShort'].isin(sampleGeneID['geneIDShort'])),:]
    sampleGeneName=notMatched2.merge(constraint,how='left',left_on=['geneName'],right_on=['gene'])
    # stack them all back together
    sample2=pd.concat([sampleTranscript,sampleGeneID,sampleGeneName],axis=0,ignore_index=True)
    sample2.loc[sample2['hg38_chr']=='X','hg38_chr']=23
    sample2.loc[sample2['hg38_chr']=='Y','hg38_chr']=24
    sample2.loc[sample2['hg38_chr']=='MT','hg38_chr']=25
    sample2['hg38_chr']=sample2['hg38_chr'].astype(int)

    # merge on the CCR data
    sample2['CCR']=np.nan
    sampleSNVs=sample2.loc[sample2['varType'].isin(['nonsynonymous SNV','synonymous SNV','stopgain','stoploss']),['hg38_chr','hg38_pos(1-based)']]
    sampleIndels=sample2.loc[sample2['varType'].isin(['frameshift insertion','frameshift deletion','frameshift substitution',
                                                    'nonframeshift insertion','nonframeshift deletion','nonframeshift substitution']),['hg38_chr','hg38_pos(1-based)','ref']]
    sampleIndels['length']=sampleIndels['ref'].str.len()
    sampleIndels['CCR']=np.nan
    sampleSNVs2=sampleSNVs.merge(CCR,how='left',left_on=['hg38_chr','hg38_pos(1-based)'],right_on=['chrom','pos']).set_index(sampleSNVs.index)
    for i in range(len(sampleIndels)):
        if i%100==0:
            print(str(i) + ' rows complete of ' + str(len(sampleIndels)))
        startPos=sampleIndels.iloc[i,1]+1
        endPos=startPos+sampleIndels.iloc[i,3]
        sampleIndels.iloc[i,4]=CCR.loc[((CCR['chrom']==sampleIndels.iloc[i,0]) & (CCR['pos'].isin(range(startPos,endPos)))),'ccr_pct'].max()
    sample2.loc[sampleSNVs2.index,'CCR']=sampleSNVs2.loc[:,'ccr_pct'].values
    sample2.loc[sampleIndels.index,'CCR']=sampleIndels.loc[:,'CCR'].values

    # merge on the pext data
    sample2['pext']=np.nan
    sampleIndels['pext']=np.nan
    sampleSNVs2=sampleSNVs.merge(pext,how='left',left_on=['hg38_chr','hg38_pos(1-based)'],right_on=['chr','pos']).set_index(sampleSNVs.index)
    for i in range(len(sampleIndels)):
        if i%100==0:
            print(str(i) + ' rows complete of ' + str(len(sampleIndels)))
        startPos=sampleIndels.iloc[i,1]+1
        endPos=startPos+sampleIndels.iloc[i,3]
        sampleIndels.iloc[i,5]=pext.loc[((pext['chr']==sampleIndels.iloc[i,0]) & (pext['pos'].isin(range(startPos,endPos)))),'pext'].max()
    sample2.loc[sampleSNVs2.index,'pext']=sampleSNVs2.loc[:,'pext'].values
    sample2.loc[sampleIndels.index,'pext']=sampleIndels.loc[:,'pext'].values

    # merge on the GERP data
    sample2['gerp']=np.nan
    sampleIndels['gerp']=np.nan
    sampleSNVs2=sampleSNVs.merge(gerp,how='left',left_on=['hg38_chr','hg38_pos(1-based)'],right_on=['chr','pos']).set_index(sampleSNVs.index)
    for i in range(len(sampleIndels)):
        if i%100==0:
            print(str(i) + ' rows complete of ' + str(len(sampleIndels)))
        startPos=sampleIndels.iloc[i,1]+1
        endPos=startPos+sampleIndels.iloc[i,3]
        sampleIndels.iloc[i,6]=gerp.loc[((gerp['chr']==sampleIndels.iloc[i,0]) & (gerp['pos'].isin(range(startPos,endPos)))),'gerp'].max()
    sample2.loc[sampleSNVs2.index,'gerp']=sampleSNVs2.loc[:,'gerp'].values
    sample2.loc[sampleIndels.index,'gerp']=sampleIndels.loc[:,'gerp'].values

    sample2=sample2.drop_duplicates(subset=['hg38_chr','hg38_pos(1-based)','ref','alt'],keep='first')
    sample2=sample2.drop(columns=['line','location','end','qual','depth','gene','transcript', 'canonical','gene_id'])
    sample2=sample2.sort_values(by=['hg38_chr','hg38_pos(1-based)','ref','alt']).reset_index(drop=True)

    # merge on GDI data
    sample2=sample2.merge(GDI,how='left',on='geneName')
    # merge on RVIS data
    sample2=sample2.merge(RVIS,how='left',on='geneName')
    
    # filtration steps that are only performed on the training and testing sets
    # remove variants within 2bp of exon-intron boundaries
    sample2=sample2.loc[~(sample2['varType'].str.contains('splicing')),:].reset_index(drop=True)
    
    # correct the sequence of proteins whose alt sequence doesn't start with an M to being non-translated
    sample2.loc[((sample2['ChangePos']==1) & (sample2['ref'].str.len()==1) & (sample2['alt'].str.len()==1) & (sample2['ref']!='-') * (sample2['alt']!='-')),'AltSeq']="*"
    
    if variantType=='dominant':
        # get rid of any variants seen in gnomAD
        sample2['controls_AF']=sample2['controls_AF'].replace('.',0).fillna(0)
        sample2=sample2.loc[sample2['controls_AF']==0,:].reset_index(drop=True)
    elif variantType=='recessive':
        # get rid of any variants seen in the homozygous state in gnomAD
        sample2['controls_nhomalt']=sample2['controls_nhomalt'].replace('.',0).fillna(0)
        sample2=sample2.loc[sample2['controls_nhomalt']==0,:].reset_index(drop=True)

    sample2=sample2.sort_values(by=['hg38_chr','hg38_pos(1-based)','ref','alt']).reset_index(drop=True)
    sample2=sample2.drop_duplicates(subset=['hg38_chr','hg38_pos(1-based)','ref','alt'],keep='first').reset_index(drop=True)
    sample2.to_csv(base + '.annotated.txt',sep='\t',index=False)
    return

In [ ]:
annotateVariantsAndFilter('clinvar_pathogenic_AR',variantType='recessive')
annotateVariantsAndFilter('clinvar_pathogenic_AD',variantType='dominant')
annotateVariantsAndFilter('clinvar_benign')

In [6]:
# remove duplicates and create validation set and training sets
AR=pd.read_csv('./clinvar_pathogenic_AR.annotated.txt',sep='\t',low_memory=False)
AD=pd.read_csv('./clinvar_pathogenic_AD.annotated.txt',sep='\t',low_memory=False)
benign=pd.read_csv('./clinvar_benign.annotated.txt',sep='\t',low_memory=False)
AR['classLabel']=2
AD['classLabel']=1
benign['classLabel']=0
allData=pd.concat([AR,AD,benign],axis=0,ignore_index=True)
allData=allData.sort_values(by=['hg38_chr','hg38_pos(1-based)','ref','alt'])
allData=allData.drop_duplicates(subset=['hg38_chr','hg38_pos(1-based)','ref','alt'],keep=False).reset_index(drop=True)
allData=allData.loc[:,['varType', 'hg38_chr', 'hg38_pos(1-based)', 'ref', 'alt', 'WildtypeSeq', 'AltSeq', 'ChangePos', 'TranscriptID', 'TranscriptIDShort',
       'geneName', 'geneID', 'geneIDShort', 'pLI', 'pNull', 'pRec', 'mis_z', 'lof_z', 'controls_AF', 'controls_nhomalt', 'CCR', 'pext', 'gerp', 'GDI', 'RVIS_ExAC_0.05', 'classLabel']]


In [3]:
allData.to_csv('./allData2025.txt',sep='\t',index=False)

In [1]:
!wc -l allData2025.txt

298074 allData2025.txt


## Validation set

In [8]:
validationSet=allData.sample(n=2000,replace=False,random_state=1).sort_values(by=['hg38_chr','hg38_pos(1-based)','ref','alt'],ascending=True)
trainingSet=allData.drop(validationSet.index).sort_values(by=['hg38_chr','hg38_pos(1-based)','ref','alt'],ascending=True)

In [9]:
trainingSet.to_csv('./trainingSet_data2025_ann2025_hg38.txt',sep='\t',index=False)
validationSet.to_csv('./validationSet_data2025_ann2025_hg38.txt',sep='\t',index=False)

In [10]:
trainingSet.loc[:,'classLabel'].value_counts()

classLabel
0    206676
2     54244
1     35153
Name: count, dtype: int64

In [11]:
validationSet.loc[:,'classLabel'].value_counts()

classLabel
0    1433
2     354
1     213
Name: count, dtype: int64

In [12]:
!wc -l trainingSet_data2025_ann2025_hg38.txt

296074 trainingSet_data2025_ann2025_hg38.txt


In [13]:
!wc -l validationSet_data2025_ann2025_hg38.txt

2001 validationSet_data2025_ann2025_hg38.txt
